In [ ]:
#Notebook para generar tiempos y resultados del algoritmo 
#para todas las posibles combinaciones del orden para 2, 3 y 4 iteraciones


from pyspark import SparkContext
import numpy as np
from funciones import *
import time as t
import itertools
sc = SparkContext()

In [ ]:
#Factores en orden para obtenerlos rapido cuando cambiemos el orden
factores_ordenados = [[0.25,0.25,0.5],[1,0.25,0.5],[1,0.25,0.5],[1,0.25,0.25]]
orden = [0,1,2,3]
ordenes_posibles = [list(x) for x in list(itertools.permutations(orden))]

In [ ]:
def algoritmo1(list_of_maps,list_of_matchs,last_match):
    start1 = t.time()
    rdd = sc.textFile('cociente_raw_uniform_od.csv').map(mapeo_desde_archivo)
    rdd_op = rdd
    for i in range(len(orden)-2):
        rdd_op = rdd_op.map(list_of_maps[i]).flatMap(separar_keys).groupByKey().flatMap(list_of_matchs[i])
    rdd_op = rdd_op.map(seleccionar_map_intermedio(orden[0],orden[i]))
    rdd_op = rdd_op.map(copiar_leading_set).flatMap(separar_keys)
    rdd_op = rdd_op.groupByKey().flatMap(last_match).map(orden_final)
    rdd_op = rdd_op.map(map_registros).groupByKey().map(merge_registros)
    result = rdd_op.collect()
    end1= t.time()
    number_with_2 = rdd_op.count()
    return start1, end1, number_with_2

In [ ]:
def algoritmo2(list_of_maps,list_of_matchs,last_match):
    start2 = t.time()
    rdd = sc.textFile('cociente_raw_uniform_od.csv').map(mapeo_desde_archivo)
    rdd_op = rdd
    for i in range(len(orden)-1):
        rdd_op = rdd_op.map(list_of_maps[i]).flatMap(separar_keys).groupByKey().flatMap(list_of_matchs[i])
    rdd_op = rdd_op.map(seleccionar_map_intermedio(orden[0],orden[i]))
    rdd_op = rdd_op.map(copiar_leading_set).flatMap(separar_keys)
    rdd_op = rdd_op.groupByKey().flatMap(last_match).map(orden_final)
    rdd_op = rdd_op.map(map_registros).groupByKey().map(merge_registros)
    result = rdd_op.collect()
    end2= t.time()
    number_with_3 = rdd_op.count()
    return start2, end2, number_with_3

In [ ]:
def algoritmo3(list_of_maps,list_of_matchs,last_match):
    start3 = t.time()
    rdd = sc.textFile('cociente_raw_uniform_od.csv').map(mapeo_desde_archivo)
    rdd_op = rdd
    for i in range(len(orden)):
        rdd_op = rdd_op.map(list_of_maps[i]).flatMap(separar_keys).groupByKey().flatMap(list_of_matchs[i])
    rdd_op = rdd_op.map(seleccionar_map_intermedio(orden[0],orden[i]))
    rdd_op = rdd_op.map(copiar_leading_set).flatMap(separar_keys)
    rdd_op = rdd_op.groupByKey().flatMap(last_match).map(orden_final)
    rdd_op = rdd_op.map(map_registros).groupByKey().map(merge_registros)
    result = rdd_op.collect()
    end3= t.time()
    number_with_4 = rdd_op.count()
    return start3, end3, number_with_4

In [ ]:
def algoritmo_completo(orden):
    longitud = len(orden)
    factores=[0 for x in range(len(orden))]
    factores_origen = [1,0.25,0.25,0.5] #Colocados en orden de los campos originales
    for i in range(len(orden)):
        factores[i] = factores_origen[:orden[i]]+factores_origen[orden[i]+1:]

    distancias=[0 for x in range(len(orden))]
    distancias_origen = [L_distance_vacios,L_distance_vacios, distancia_levenstein_separando_vacios,L_distance_vacios]
    for i in range(len(orden)):
        distancias[i] = distancias_origen[:orden[i]]+distancias_origen[orden[i]+1:]
    multiplicador = 2/3
    for i in range(len(orden)):
        factores[i] = [multiplicador*x for x in factores[i]]

    list_of_maps = [seleccionar_map(orden[0])]
    list_of_matchs = []
    for i in range(len(orden)-1):
        list_of_maps.append(seleccionar_map_intermedio(orden[i+1],orden[i]))
    for i in range(len(orden)):
        list_of_matchs.append(seleccionar_match(factores[i],distancias[i]))
    last_match = seleccionar_match(factores[0],distancias[0], excluyentes = 2) 
    
    rdd = sc.textFile('cociente_raw_uniform_od.csv').map(mapeo_desde_archivo)
    initial_number= rdd.count()
    start1, end1, number_with_2 = algoritmo1(list_of_maps,list_of_matchs,last_match)
    start2, end2, number_with_3 = algoritmo2(list_of_maps,list_of_matchs,last_match)
    start3, end3, number_with_4 = algoritmo3(list_of_maps,list_of_matchs,last_match)
    print('En esta ejecución el orden es: {}'.format(orden))
    print('Inicialmente teniamos: {} entradas'.format(initial_number))
    print('Con 2 elementos de key tenemos: {} entradas'.format(number_with_2))
    print('Con 2 elementos de key cazamos {} entradas que se pueden agrupar. Un {}% del total'.format(initial_number-number_with_2,(initial_number-number_with_2)*100/initial_number))
    print('El algoritmo con 2 elementos tarda {} segundos'.format(end1-start1))
    print('Con 3 elementos de key tenemos: {} entradas'.format(number_with_3))
    print('Con 3 elementos de key cazamos {} entradas que se pueden agrupar. Un {}% del total'.format(initial_number-number_with_3,(initial_number-number_with_3)*100/initial_number))
    print('El algoritmo con 3 elementos tarda {} segundos'.format(end2-start2))
    print('Con 4 elementos de key tenemos: {} entradas'.format(number_with_4))
    print('Con 4 elementos de key cazamos {} entradas que se pueden agrupar. Un {}% del total'.format(initial_number-number_with_4,(initial_number-number_with_4)*100/initial_number))
    print('El algoritmo con 4 elementos tarda {} segundos'.format(end3-start3))

In [ ]:
for orden in ordenes_posibles:
    algoritmo_completo(orden)
    print()

In [ ]:
#Parece que con más vueltas apenas estamos mejorando los resultados obtenidos

En esta ejecución el orden es: [0, 1, 2, 3]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23445 entradas
Con 2 elementos de key cazamos 11839 entradas que se pueden agrupar. Un 33.55345198957034% del total
El algoritmo con 2 elementos tarda 74.34843015670776 segundos
Con 3 elementos de key tenemos: 23311 entradas
Con 3 elementos de key cazamos 11973 entradas que se pueden agrupar. Un 33.9332275252239% del total
El algoritmo con 3 elementos tarda 100.08595299720764 segundos
Con 4 elementos de key tenemos: 23298 entradas
Con 4 elementos de key cazamos 11986 entradas que se pueden agrupar. Un 33.97007142047387% del total
El algoritmo con 4 elementos tarda 103.70276808738708 segundos

En esta ejecución el orden es: [0, 1, 3, 2]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23445 entradas
Con 2 elementos de key cazamos 11839 entradas que se pueden agrupar. Un 33.55345198957034% del total
El algoritmo con 2 elementos tarda 72.18885779380798 segundos
Con 3 elementos de key tenemos: 23298 entradas
Con 3 elementos de key cazamos 11986 entradas que se pueden agrupar. Un 33.97007142047387% del total
El algoritmo con 3 elementos tarda 82.49238753318787 segundos
Con 4 elementos de key tenemos: 23298 entradas
Con 4 elementos de key cazamos 11986 entradas que se pueden agrupar. Un 33.97007142047387% del total
El algoritmo con 4 elementos tarda 119.62910008430481 segundos

En esta ejecución el orden es: [0, 2, 1, 3]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23542 entradas
Con 2 elementos de key cazamos 11742 entradas que se pueden agrupar. Un 33.278539848089785% del total
El algoritmo con 2 elementos tarda 90.40289354324341 segundos
Con 3 elementos de key tenemos: 23309 entradas
Con 3 elementos de key cazamos 11975 entradas que se pueden agrupar. Un 33.938895816800816% del total
El algoritmo con 3 elementos tarda 96.63458251953125 segundos
Con 4 elementos de key tenemos: 23298 entradas
Con 4 elementos de key cazamos 11986 entradas que se pueden agrupar. Un 33.97007142047387% del total
El algoritmo con 4 elementos tarda 107.85621333122253 segundos

En esta ejecución el orden es: [0, 2, 3, 1]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23542 entradas
Con 2 elementos de key cazamos 11742 entradas que se pueden agrupar. Un 33.278539848089785% del total
El algoritmo con 2 elementos tarda 86.7055389881134 segundos
Con 3 elementos de key tenemos: 23305 entradas
Con 3 elementos de key cazamos 11979 entradas que se pueden agrupar. Un 33.95023239995465% del total
El algoritmo con 3 elementos tarda 99.04949808120728 segundos
Con 4 elementos de key tenemos: 23298 entradas
Con 4 elementos de key cazamos 11986 entradas que se pueden agrupar. Un 33.97007142047387% del total
El algoritmo con 4 elementos tarda 99.67367362976074 segundos

En esta ejecución el orden es: [0, 3, 1, 2]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23488 entradas
Con 2 elementos de key cazamos 11796 entradas que se pueden agrupar. Un 33.43158372066659% del total
El algoritmo con 2 elementos tarda 70.74175477027893 segundos
Con 3 elementos de key tenemos: 23298 entradas
Con 3 elementos de key cazamos 11986 entradas que se pueden agrupar. Un 33.97007142047387% del total
El algoritmo con 3 elementos tarda 78.91798090934753 segundos
Con 4 elementos de key tenemos: 23298 entradas
Con 4 elementos de key cazamos 11986 entradas que se pueden agrupar. Un 33.97007142047387% del total
El algoritmo con 4 elementos tarda 108.83762884140015 segundos

En esta ejecución el orden es: [0, 3, 2, 1]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23488 entradas
Con 2 elementos de key cazamos 11796 entradas que se pueden agrupar. Un 33.43158372066659% del total
El algoritmo con 2 elementos tarda 71.346688747406 segundos
Con 3 elementos de key tenemos: 23306 entradas
Con 3 elementos de key cazamos 11978 entradas que se pueden agrupar. Un 33.947398254166195% del total
El algoritmo con 3 elementos tarda 109.61439633369446 segundos
Con 4 elementos de key tenemos: 23298 entradas
Con 4 elementos de key cazamos 11986 entradas que se pueden agrupar. Un 33.97007142047387% del total
El algoritmo con 4 elementos tarda 104.04324126243591 segundos

En esta ejecución el orden es: [1, 0, 2, 3]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23448 entradas
Con 2 elementos de key cazamos 11836 entradas que se pueden agrupar. Un 33.54494955220497% del total
El algoritmo con 2 elementos tarda 40.560791015625 segundos
Con 3 elementos de key tenemos: 23311 entradas
Con 3 elementos de key cazamos 11973 entradas que se pueden agrupar. Un 33.9332275252239% del total
El algoritmo con 3 elementos tarda 58.810539960861206 segundos
Con 4 elementos de key tenemos: 23298 entradas
Con 4 elementos de key cazamos 11986 entradas que se pueden agrupar. Un 33.97007142047387% del total
El algoritmo con 4 elementos tarda 59.31295108795166 segundos

En esta ejecución el orden es: [1, 0, 3, 2]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23448 entradas
Con 2 elementos de key cazamos 11836 entradas que se pueden agrupar. Un 33.54494955220497% del total
El algoritmo con 2 elementos tarda 39.29196071624756 segundos
Con 3 elementos de key tenemos: 23298 entradas
Con 3 elementos de key cazamos 11986 entradas que se pueden agrupar. Un 33.97007142047387% del total
El algoritmo con 3 elementos tarda 40.5097439289093 segundos
Con 4 elementos de key tenemos: 23298 entradas
Con 4 elementos de key cazamos 11986 entradas que se pueden agrupar. Un 33.97007142047387% del total
El algoritmo con 4 elementos tarda 61.48671269416809 segundos

En esta ejecución el orden es: [1, 2, 0, 3]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23504 entradas
Con 2 elementos de key cazamos 11780 entradas que se pueden agrupar. Un 33.38623738805124% del total
El algoritmo con 2 elementos tarda 24.204564094543457 segundos
Con 3 elementos de key tenemos: 23309 entradas
Con 3 elementos de key cazamos 11975 entradas que se pueden agrupar. Un 33.938895816800816% del total
El algoritmo con 3 elementos tarda 75.25827598571777 segundos
Con 4 elementos de key tenemos: 23298 entradas
Con 4 elementos de key cazamos 11986 entradas que se pueden agrupar. Un 33.97007142047387% del total
El algoritmo con 4 elementos tarda 71.95004057884216 segundos

En esta ejecución el orden es: [1, 2, 3, 0]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23504 entradas
Con 2 elementos de key cazamos 11780 entradas que se pueden agrupar. Un 33.38623738805124% del total
El algoritmo con 2 elementos tarda 24.694889783859253 segundos
Con 3 elementos de key tenemos: 23302 entradas
Con 3 elementos de key cazamos 11982 entradas que se pueden agrupar. Un 33.95873483732003% del total
El algoritmo con 3 elementos tarda 25.90848684310913 segundos
Con 4 elementos de key tenemos: 23298 entradas
Con 4 elementos de key cazamos 11986 entradas que se pueden agrupar. Un 33.97007142047387% del total
El algoritmo con 4 elementos tarda 69.8276526927948 segundos

En esta ejecución el orden es: [1, 3, 0, 2]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23399 entradas
Con 2 elementos de key cazamos 11885 entradas que se pueden agrupar. Un 33.683822695839474% del total
El algoritmo con 2 elementos tarda 8.17535138130188 segundos
Con 3 elementos de key tenemos: 23299 entradas
Con 3 elementos de key cazamos 11985 entradas que se pueden agrupar. Un 33.96723727468541% del total
El algoritmo con 3 elementos tarda 50.739078998565674 segundos
Con 4 elementos de key tenemos: 23298 entradas
Con 4 elementos de key cazamos 11986 entradas que se pueden agrupar. Un 33.97007142047387% del total
El algoritmo con 4 elementos tarda 71.99037194252014 segundos

En esta ejecución el orden es: [1, 3, 2, 0]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23399 entradas
Con 2 elementos de key cazamos 11885 entradas que se pueden agrupar. Un 33.683822695839474% del total
El algoritmo con 2 elementos tarda 7.2316248416900635 segundos
Con 3 elementos de key tenemos: 23302 entradas
Con 3 elementos de key cazamos 11982 entradas que se pueden agrupar. Un 33.95873483732003% del total
El algoritmo con 3 elementos tarda 32.68043088912964 segundos
Con 4 elementos de key tenemos: 23298 entradas
Con 4 elementos de key cazamos 11986 entradas que se pueden agrupar. Un 33.97007142047387% del total
El algoritmo con 4 elementos tarda 89.28824782371521 segundos

En esta ejecución el orden es: [2, 0, 1, 3]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23409 entradas
Con 2 elementos de key cazamos 11875 entradas que se pueden agrupar. Un 33.65548123795488% del total
El algoritmo con 2 elementos tarda 93.4469485282898 segundos
Con 3 elementos de key tenemos: 23184 entradas
Con 3 elementos de key cazamos 12100 entradas que se pueden agrupar. Un 34.293164040358235% del total
El algoritmo con 3 elementos tarda 95.48848581314087 segundos
Con 4 elementos de key tenemos: 23176 entradas
Con 4 elementos de key cazamos 12108 entradas que se pueden agrupar. Un 34.31583720666591% del total
El algoritmo con 4 elementos tarda 97.28845047950745 segundos

En esta ejecución el orden es: [2, 0, 3, 1]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23409 entradas
Con 2 elementos de key cazamos 11875 entradas que se pueden agrupar. Un 33.65548123795488% del total
El algoritmo con 2 elementos tarda 91.65402269363403 segundos
Con 3 elementos de key tenemos: 23183 entradas
Con 3 elementos de key cazamos 12101 entradas que se pueden agrupar. Un 34.2959981861467% del total
El algoritmo con 3 elementos tarda 100.8512282371521 segundos
Con 4 elementos de key tenemos: 23174 entradas
Con 4 elementos de key cazamos 12110 entradas que se pueden agrupar. Un 34.32150549824283% del total
El algoritmo con 4 elementos tarda 114.54870939254761 segundos

En esta ejecución el orden es: [2, 1, 0, 3]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23389 entradas
Con 2 elementos de key cazamos 11895 entradas que se pueden agrupar. Un 33.71216415372407% del total
El algoritmo con 2 elementos tarda 65.89159822463989 segundos
Con 3 elementos de key tenemos: 23193 entradas
Con 3 elementos de key cazamos 12091 entradas que se pueden agrupar. Un 34.267656728262104% del total
El algoritmo con 3 elementos tarda 115.86371278762817 segundos
Con 4 elementos de key tenemos: 23176 entradas
Con 4 elementos de key cazamos 12108 entradas que se pueden agrupar. Un 34.31583720666591% del total
El algoritmo con 4 elementos tarda 114.34143090248108 segundos

En esta ejecución el orden es: [2, 1, 3, 0]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23389 entradas
Con 2 elementos de key cazamos 11895 entradas que se pueden agrupar. Un 33.71216415372407% del total
El algoritmo con 2 elementos tarda 61.47028875350952 segundos
Con 3 elementos de key tenemos: 23188 entradas
Con 3 elementos de key cazamos 12096 entradas que se pueden agrupar. Un 34.2818274572044% del total
El algoritmo con 3 elementos tarda 69.54671788215637 segundos
Con 4 elementos de key tenemos: 23174 entradas
Con 4 elementos de key cazamos 12110 entradas que se pueden agrupar. Un 34.32150549824283% del total
El algoritmo con 4 elementos tarda 114.3285505771637 segundos

En esta ejecución el orden es: [2, 3, 0, 1]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23358 entradas
Con 2 elementos de key cazamos 11926 entradas que se pueden agrupar. Un 33.80002267316631% del total
El algoritmo con 2 elementos tarda 68.81416392326355 segundos
Con 3 elementos de key tenemos: 23182 entradas
Con 3 elementos de key cazamos 12102 entradas que se pueden agrupar. Un 34.29883233193515% del total
El algoritmo con 3 elementos tarda 100.45373201370239 segundos
Con 4 elementos de key tenemos: 23173 entradas
Con 4 elementos de key cazamos 12111 entradas que se pueden agrupar. Un 34.32433964403129% del total
El algoritmo con 4 elementos tarda 102.10277819633484 segundos

En esta ejecución el orden es: [2, 3, 1, 0]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23358 entradas
Con 2 elementos de key cazamos 11926 entradas que se pueden agrupar. Un 33.80002267316631% del total
El algoritmo con 2 elementos tarda 62.34086537361145 segundos
Con 3 elementos de key tenemos: 23181 entradas
Con 3 elementos de key cazamos 12103 entradas que se pueden agrupar. Un 34.301666477723614% del total
El algoritmo con 3 elementos tarda 62.34489417076111 segundos
Con 4 elementos de key tenemos: 23174 entradas
Con 4 elementos de key cazamos 12110 entradas que se pueden agrupar. Un 34.32150549824283% del total
El algoritmo con 4 elementos tarda 108.71320343017578 segundos

En esta ejecución el orden es: [3, 0, 1, 2]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23377 entradas
Con 2 elementos de key cazamos 11907 entradas que se pueden agrupar. Un 33.74617390318558% del total
El algoritmo con 2 elementos tarda 48.65224838256836 segundos
Con 3 elementos de key tenemos: 23187 entradas
Con 3 elementos de key cazamos 12097 entradas que se pueden agrupar. Un 34.284661602992855% del total
El algoritmo con 3 elementos tarda 41.878660440444946 segundos
Con 4 elementos de key tenemos: 23190 entradas
Con 4 elementos de key cazamos 12094 entradas que se pueden agrupar. Un 34.27615916562748% del total
El algoritmo con 4 elementos tarda 69.44616746902466 segundos

En esta ejecución el orden es: [3, 0, 2, 1]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23377 entradas
Con 2 elementos de key cazamos 11907 entradas que se pueden agrupar. Un 33.74617390318558% del total
El algoritmo con 2 elementos tarda 38.453081130981445 segundos
Con 3 elementos de key tenemos: 23201 entradas
Con 3 elementos de key cazamos 12083 entradas que se pueden agrupar. Un 34.24498356195443% del total
El algoritmo con 3 elementos tarda 63.80980181694031 segundos
Con 4 elementos de key tenemos: 23189 entradas
Con 4 elementos de key cazamos 12095 entradas que se pueden agrupar. Un 34.27899331141594% del total
El algoritmo con 4 elementos tarda 61.580148220062256 segundos

En esta ejecución el orden es: [3, 1, 0, 2]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23294 entradas
Con 2 elementos de key cazamos 11990 entradas que se pueden agrupar. Un 33.981408003627706% del total
El algoritmo con 2 elementos tarda 7.602078914642334 segundos
Con 3 elementos de key tenemos: 23187 entradas
Con 3 elementos de key cazamos 12097 entradas que se pueden agrupar. Un 34.284661602992855% del total
El algoritmo con 3 elementos tarda 46.11801266670227 segundos
Con 4 elementos de key tenemos: 23191 entradas
Con 4 elementos de key cazamos 12093 entradas que se pueden agrupar. Un 34.27332501983902% del total
El algoritmo con 4 elementos tarda 68.66779661178589 segundos

En esta ejecución el orden es: [3, 1, 2, 0]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23294 entradas
Con 2 elementos de key cazamos 11990 entradas que se pueden agrupar. Un 33.981408003627706% del total
El algoritmo con 2 elementos tarda 7.258602619171143 segundos
Con 3 elementos de key tenemos: 23198 entradas
Con 3 elementos de key cazamos 12086 entradas que se pueden agrupar. Un 34.25348599931981% del total
El algoritmo con 3 elementos tarda 28.1044704914093 segundos
Con 4 elementos de key tenemos: 23188 entradas
Con 4 elementos de key cazamos 12096 entradas que se pueden agrupar. Un 34.2818274572044% del total
El algoritmo con 4 elementos tarda 79.60277438163757 segundos

En esta ejecución el orden es: [3, 2, 0, 1]
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23454 entradas
Con 2 elementos de key cazamos 11830 entradas que se pueden agrupar. Un 33.52794467747421% del total
El algoritmo con 2 elementos tarda 28.43150782585144 segundos
Con 3 elementos de key tenemos: 23200 entradas
Con 3 elementos de key cazamos 12084 entradas que se pueden agrupar. Un 34.24781770774289% del total
El algoritmo con 3 elementos tarda 74.13491797447205 segundos
Con 4 elementos de key tenemos: 23186 entradas
Con 4 elementos de key cazamos 12098 entradas que se pueden agrupar. Un 34.28749574878132% del total
El algoritmo con 4 elementos tarda 77.76853513717651 segundos

En esta ejecución el orden es: [3, 2, 1, 0] 
Inicialmente teniamos: 35284 entradas
Con 2 elementos de key tenemos: 23454 entradas
Con 2 elementos de key cazamos 11830 entradas que se pueden agrupar. Un 33.52794467747421% del total
El algoritmo con 2 elementos tarda 28.372629642486572 segundos
Con 3 elementos de key tenemos: 23192 entradas
Con 3 elementos de key cazamos 12092 entradas que se pueden agrupar. Un 34.27049087405056% del total
El algoritmo con 3 elementos tarda 29.637075901031494 segundos
Con 4 elementos de key tenemos: 23186 entradas
Con 4 elementos de key cazamos 12098 entradas que se pueden agrupar. Un 34.28749574878132% del total
El algoritmo con 4 elementos tarda 69.78548836708069 segundos

